In [ ]:
	Set PatientID = $ORDER(^PAPERi("PAPMI_PatNo",PatNo,0))
	Set AdmType = 0
	For {
		Set AdmType = $o(^PAPERdr(PatientID,"ADM",AdmType))
		q:AdmType=""
		Set EpisodeID = 0
		For {
			Set EpisodeID = $o(^PAPERdr(PatientID,"ADM",AdmType,EpisodeID))
			Quit:EpisodeID=""
			Set ord = 0
			For {
				Set ord = $o(^OEORD(0,"Adm",EpisodeID,ord))
				Quit:ord=""
				Set item = 0
				For {
					Set item = $o(^OEORD(ord,"I",item))
		            Quit:item=""
					s stat=$p($g(^OEORD(ord,"I",item,1)),"^",13)
					s:stat'="" stat = $p(^OEC("OSTAT",stat),"^",1)
					continue:(stat'="E")&&(stat'="V")
					s stt = $p($g(^OEORD(ord,"I",item,1)),"^",9)
					s date = $p($g(^OEORD(ord,"I",item,3)),"^",7)
					s time = $p($g(^OEORD(ord,"I",item,3)),"^",15)
					s loc = $p($g(^OEORD(ord,"I",item,1)),"^",3)
					s arcim = $p($g(^OEORD(ord,"I",item,1)),"^",2)
					s cat = $p($g(^ARCIM(+arcim,1,1)),"^",10)
		            Set exe = 0
		            For {
		                Set exe = $o(^OEORD(ord,"I",item,"X",exe))
		                Quit:exe=""
						s exeStat = $p($g(^OEORD(ord,"I",item,"X",exe)),"^",16)
		                s:exeStat'="" exeStatCode = $p($g(^OEC("STAT",exeStatusId)),"^",1)
		                continue:$g(exeStatCode)'="F"
		            }
				}
			}
		}
	}


In [ ]:
/**
	* @debug w ##class(web.DHCDocMain).Pinyin("魏方茜")
*/
ClassMethod Pinyin(Name)
{
	q:Name="" ""
	s out = ""
	s name = ##class(web.DHCINSUPort).GetCNCODE(Name,3," ")
	for i=1:1:$l(name," ") {
		s one = $p(name," ",i)
		s one = $e(one,1,1) _ $zcvt($e(one,2,$l(one)),"l")
		if out = "" {
			s out = one	
		}
		else {
			s out = out _ " " _ one	
		}
	}
	q out
}

In [ ]:
;获取就诊医院
s HospID=##class(DHCDoc.Common.Hospital).GetAdmHospitalId(EpisodeId)

In [ ]:
;获取默认费别
w ##class(web.DHCDocOrderCommon).GetDefaultPrescriptType(EpisodeID,"")

;获取所有费别
##class(web.DHCDocOrderCommon).GetPrescriptTypeInfo()

In [ ]:
Query OPOrd(start, end) As websys.Query(ROWSPEC = "写门诊病历人次:%String,未写门诊病历人次:%String,有医嘱费用人次:%String,无医嘱费用人次:%String") [ SqlProc ]
{
}

/// debug:	d ##class(%ResultSet).RunQuery("web.DHCDocMain", "OPOrd")
/// call web.dhcdocmain_opord('2025-01-01','2025-01-04');
ClassMethod OPOrdExecute(ByRef qHandle As %Binary, start, end) As %Status
{
	s repid = $i(^CacheTemp)
	s ind = 1
	s qHandle = $lb(0, repid, 0)
	;s languageId = ..%LanguageID()
	s start = $case($g(start),"":+$h,:$zdh(start,3))
	s end = $case($g(end),"":+$h,:$zdh(end,3))
	for date=start:1:end {
		s EpisodeID = 0
		for {
			s EpisodeID = $o(^PAADMi("PAADM_AdmDate",date,EpisodeID))
			q:EpisodeID=""
			s AdmType = $p($g(^PAADM(EpisodeID)),"^",2)
			continue:AdmType="I"
			s admSum = $g(admSum) + 1
			s MZRecordStatus=##class(DHCDoc.Interface.Inside.InvokeOPAdm).GetMZRecordStatus(EpisodeID)
			s:MZRecordStatus="已签" hasRecord = $g(hasRecord) + 1
			s flag = 0
			Set ord = 0
			For {
				Set ord = $o(^OEORD(0,"Adm",EpisodeID,ord))
				Quit:ord=""
				Set item = 0
				For {
					Set item = $o(^OEORD(ord,"I",item))
		            Quit:item=""
					s stat=$p($g(^OEORD(ord,"I",item,1)),"^",13)
					s:stat'="" stat = $p(^OEC("OSTAT",stat),"^",1)
					continue:(stat'="E")&&(stat'="V")
					s arcim = $p($g(^OEORD(ord,"I",item,1)),"^",2)
					s cat = $p($g(^ARCIM(+arcim,1,1)),"^",1)
		            s flag = 1
				}
			}
			s:flag=1 hasOrd = $g(hasOrd)+1
		}
	}
	d OutputRow
	Quit $$$OK

OutputRow
	s Data = $lb($g(hasRecord), $g(admSum) - $g(hasRecord), $g(hasOrd), $g(admSum) - $g(hasOrd))
	s ^CacheTemp(repid, ind) = Data
	s ind = ind + 1
	q
}